In [2]:
!pip install estnltk==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83.4 MB 53 kB/s 
     |████████████████████████████████| 965 kB 55.9 MB/s 
     |████████████████████████████████| 195 kB 55.2 MB/s 
     |████████████████████████████████| 106 kB 57.2 MB/s 
     |████████████████████████████████| 1.6 MB 51.2 MB/s 
     |████████████████████████████████| 15.7 MB 26.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [3]:
import pandas as pd
import re
from estnltk import Text

texts = pd.read_csv('texts_about_skills.csv')

In [27]:
texts

,text_id,text
0,text0,"Võistlusülesande lahendamiseks vajalikud erialased teadmised ja oskused: elektroonikakomponentide formeerimine ja paigaldamine trükkplaadile, elektroonikakomponentide trükkplaadilt maha jootmine, elektroonikakomponentide trükkplaadile jootmine, jooteseadmete kasutamine, jootematerjalide kasutamine, elektroonikakomponentide ohutus."
1,text1,Võistlusülesandena tuli elektroonikutel etteantud komponentidest kokku monteerida pindmontaaž- ja läbiviiktehnoloogiaid kasutav elektroonikaseadme trükkplaat.
2,text2,"See on väga hea ja positiivne alus, kust kõrgemaid probleemilahenduse ja tehnoloogia kasutuse oskusi edasi arendada."
3,text3,"“Töövõtted ja ohutustehnika olid kõigil võistlejatel selged ja nende eest said kõik maksimumpunktid,” tunnustas Kolk."
4,text4,"Enamasti on nendeks pigem eraisikud ja väikese suurusega ettevõtjad, kel tihti puudub ka oskus, võimekus ja eelarve oma dokumentide pikaajalise turvalisuse tagamiseks."
...,...,...
321,text321,"Ettevõtjate hinnangul vajavad koolilõpetajate puhul arendamist jooniste lugemise, ruumilise mõtlemise ja montaažioskused."
322,text322,"Ettevõtjate hinnangul vajavad ka siin koolilõpetajate puhul arendamist jooniste lugemise, ruumilise mõtlemise ja montaažioskused."
323,text323,Erialastest oskustest vajavad arendamist jooniste lugemise ja montaažioskused.
324,text324,"Olulisemaks muutub oskus töötada samaaegselt erinevate seadmetega, kuna IKT-lahenduste kasutuselevõtt võimaldab ühel inimesel töötada korraga mitmel tööpingil."


In [28]:
raw_text = texts['text'].values
ids = texts['text_id'].values
raw_text[2]

'See on väga hea ja positiivne alus, kust kõrgemaid probleemilahenduse ja tehnoloogia kasutuse oskusi edasi arendada. '

##Loading stopword files in

In [5]:
# opening files in read mode
stopwords = open("estonian-stopwords.txt", "r", encoding="utf-8")
stopwords_lemmas = open("estonian-stopwords-lemmas.txt", "r", encoding="utf-8")
  
# reading files
stop_data = stopwords.read()
stop_lemma_data = stopwords_lemmas.read()
  
# replacing end splitting the text when newline ('\n') is seen.
stop = stop_data.split("\n")
stop_lemmas = stop_lemma_data.split("\n")

#close the files
stopwords.close()
stopwords_lemmas.close()

##Lemmatized, punctuation separate from word, all lowercase

In [6]:
lemmatized_text = [] #here are the texts in lemmatized fashion, punctuation is separated from the word, all lowercase 
lemmas_lists = [] #here are lists of all the lemmatized words per text, each word (including punct) is an element in the list

for text in raw_text:
  string = ""
  lemmas = []
  txt = Text(text)
  txt.tag_layer()
  for word in txt.words: #by word
    string += word.lemma[0] + " " #can have multiple lemmas, but lets just take the first for simplicity
    lemmas.append(word.lemma[0])
  lemmatized_text.append(string)
  lemmas_lists.append(lemmas)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
lemmatized_text[2]

'see olema väga hea ja positiivne alus , kust kõrgem probleemilahendus ja tehnoloogia kasutus oskus edasi arendama . '

In [29]:
lemmas_lists[2]

['see',
 'olema',
 'väga',
 'hea',
 'ja',
 'positiivne',
 'alus',
 ',',
 'kust',
 'kõrgem',
 'probleemilahendus',
 'ja',
 'tehnoloogia',
 'kasutus',
 'oskus',
 'edasi',
 'arendama',
 '.']

In [32]:
lemma_df = pd.DataFrame(list(zip(ids, lemmas_lists)),
               columns =['text_id', 'lemma_list'])
lemma_df.to_csv('lemmas.csv', index=False)

##Same as previous + stopwords removed

In [34]:
lemmatized_stop_text = [] #here are the lemmatized texts, punctuation is separated from the word, all lowercase, stopwords removed
lemmas_stop_lists = [] #here are lists of all the lemmatized words per text, each word (including punct) is an element in the list, stopwords removed

for text in raw_text:
  string = ""
  lemmas = []
  txt = Text(text)
  txt.tag_layer()
  for word in txt.words: #by word
    if word.lemma[0] not in stop_lemmas:
      string += word.lemma[0] + " " #can have multiple lemmas, but lets just take the first for simplicity
      lemmas.append(word.lemma[0])
  lemmatized_stop_text.append(string)
  lemmas_stop_lists.append(lemmas)

In [35]:
lemmatized_stop_text[2]

'hea positiivne alus , kõrgem probleemilahendus tehnoloogia kasutus oskus arendama . '

In [36]:
lemmas_stop_lists[2]

['hea',
 'positiivne',
 'alus',
 ',',
 'kõrgem',
 'probleemilahendus',
 'tehnoloogia',
 'kasutus',
 'oskus',
 'arendama',
 '.']

In [38]:
lemma_stop_df = pd.DataFrame(list(zip(ids, lemmas_stop_lists)),
               columns =['text_id', 'lemma_stop_list'])
lemma_stop_df.to_csv('lemmas_stopwords.csv', index=False)

##Punctuation separated from word, not lemmatized, everything besides abbreviations and names is in lowercase

In [18]:
from estnltk.taggers import NerTagger
ner_tagger = NerTagger()

punctsplit_text = [] #here are the texts where punctuation is separated from the word
punctsplit_lists = [] #here are lists of all the words per text

for text in raw_text:
  string = ""
  words = []
  txt = Text(text)
  txt.tag_layer()
  ner_tagger.tag(txt)
  for word in txt.words: #by word
    if word.partofspeech[0] == 'Y' or word.text in txt.ner.text: #if word is an abbreviation or name
      string += word.text + " " #don't put it into lowercase
      words.append(word.text)
    else:
      string += word.text.lower() + " "
      words.append(word.text.lower())
  punctsplit_text.append(string)
  punctsplit_lists.append(words)

In [19]:
punctsplit_text[7]

'LIKTA kutsub inimesi kõigis kolmes Balti riigis üles osalema projekti “ virtual internships in the baltics ” raames tasuta digioskuste kursustel , pakkudes kokku 30 000 inimesele võimalust tasuta oma teadmisi parandada ja seeläbi tuleviku karjäärivõimalusi edendada . '

In [20]:
punctsplit_lists[7]

['LIKTA',
 'kutsub',
 'inimesi',
 'kõigis',
 'kolmes',
 'Balti',
 'riigis',
 'üles',
 'osalema',
 'projekti',
 '“',
 'virtual',
 'internships',
 'in',
 'the',
 'baltics',
 '”',
 'raames',
 'tasuta',
 'digioskuste',
 'kursustel',
 ',',
 'pakkudes',
 'kokku',
 '30 000',
 'inimesele',
 'võimalust',
 'tasuta',
 'oma',
 'teadmisi',
 'parandada',
 'ja',
 'seeläbi',
 'tuleviku',
 'karjäärivõimalusi',
 'edendada',
 '.']

In [40]:
raw_df = pd.DataFrame(list(zip(ids, punctsplit_lists)),
               columns =['text_id', 'list'])
raw_df.to_csv('preprocessed.csv', index=False)

##Removed stopwords, not lemmatized, everything besides abbreviations and names is in lowercase


In [22]:
punctsplit_stop_text = [] #here are the texts where punctuation is separated from the word
punctsplit_stop_lists = [] #here are lists of all the words per text
ner_tagger = NerTagger()

for text in raw_text:
  string = ""
  words = []
  txt = Text(text)
  txt.tag_layer()
  ner_tagger.tag(txt)
  for word in txt.words: #by word
    if word.text.lower() not in stop:
      if word.partofspeech[0] == 'Y' or word.text in txt.ner.text: #if word is an abbreviation or name
        string += word.text + " " #don't put it into lowercase
        words.append(word.text)
      else:
        string += word.text.lower() + " "
        words.append(word.text.lower())
  punctsplit_stop_text.append(string)
  punctsplit_stop_lists.append(words)

In [26]:
punctsplit_stop_text[2]

'hea positiivne alus , kõrgemaid probleemilahenduse tehnoloogia kasutuse oskusi arendada . '

In [25]:
punctsplit_stop_lists[2]

['hea',
 'positiivne',
 'alus',
 ',',
 'kõrgemaid',
 'probleemilahenduse',
 'tehnoloogia',
 'kasutuse',
 'oskusi',
 'arendada',
 '.']

In [41]:
raw_stop_df = pd.DataFrame(list(zip(ids, punctsplit_stop_lists)),
               columns =['text_id', 'stop_list'])
raw_stop_df.to_csv('preprocessed_stopwords.csv', index=False)